In [ ]:
!wget -c http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip -o annotations_trainval2017.zip

In [ ]:
import re
import json
import pandas as pd

img_to_check=[]
KEYWORDS=['garden','park[s ]','landscape','view','sand[sy ]','grass[sy ]','snow[sy ]','wind[sy ]','mountain','valley','waterfall','forest','city','hill','valley','^sky', ' sky ','cliff',' ridge','mist[sy ]','cloud[sy ]','lake','countryside','sea','ocean','beach']
EXCLUSION_KEYWORDS=['sheep','banana','orange','kitchen','overhead','pizza','sandwich','plate','side','rear','individual','dog','cat','zebra','people','couple','plane','man', 'men','giraffe','adult','tourist','elephant', 'bear','boy','kid','bird','child','horse','food','person','toilet','sheep','guy','animal','girl','toddler','cow','male','young','one','two','three','herd','lamb','fruit','room','passenger','gull', 'swan','skier']
with open('./annotations/captions_val2017.json') as json_file:
    data = json.load(json_file)

df= pd.DataFrame(data['annotations'])
df2=df[df['caption'].str.contains('|'.join(KEYWORDS),flags=re.IGNORECASE) & ~ df['caption'].str.contains('|'.join(EXCLUSION_KEYWORDS),flags=re.IGNORECASE) ]
df2

img_to_check.extend([(k['coco_url'],k['id']) for k in data['images'] if k['id'] in set(df2['image_id'])])


with open('./annotations/captions_train2017.json') as json_file:
    data = json.load(json_file)

df= pd.DataFrame(data['annotations'])
df3=df[df['caption'].str.contains('|'.join(KEYWORDS),flags=re.IGNORECASE) & ~ df['caption'].str.contains('|'.join(EXCLUSION_KEYWORDS),flags=re.IGNORECASE) ]
df3

img_to_check2=((k['coco_url'],k['id']) for k in data['images'] if k['id'] in set(df3['image_id']))

In [ ]:
df_val_pair=df2.drop_duplicates(subset = ['image_id'], keep = 'first') 
df_val_pair["file_name"]=df_val_pair.apply(lambda x: str(x.image_id).zfill(12)+'.jpg', axis=1)
df_val_pair=df_val_pair.reset_index().set_index("file_name")
df_val_pair=df_val_pair.drop(columns=['index','id','image_id'])
df_val_pair

df_train_pair=df3.drop_duplicates(subset = ['image_id'], keep = 'first') 
df_train_pair["file_name"]=df_train_pair.apply(lambda x: str(x.image_id).zfill(12)+'.jpg', axis=1)
df_train_pair=df_train_pair.reset_index().set_index("file_name")
df_train_pair=df_train_pair.drop(columns=['index','id','image_id'])
df_train_pair

In [ ]:
df_val_pair.to_csv("validation.csv")
df_train_pair.to_csv("train.csv")

In [ ]:
import random
import time
from IPython.display import Image, display, clear_output
from ipywidgets import widgets

queue=img_to_check
result=[]
done=[]
def choose():
    
    if queue:
      (im,imgid)=queue.pop()
      
      if imgid in done:
        choose()
      else:
      
        button1 = widgets.Button(description = "keep")
        button2 = widgets.Button(description = "drop")

        container = widgets.HBox(children=[button1,button2])
        


        def on_button1_clicked(b):
            done.append(imgid)
            result.append(imgid)
            container.close()
            clear_output()
            choose()

        def on_button2_clicked(b):
            done.append(imgid)
            container.close()
            clear_output()
            choose()


        button1.on_click(on_button1_clicked)
        button2.on_click(on_button2_clicked)
        display(Image(im))
        display(container)

choose() # initializes the first choice
print(result)

In [ ]:
!pip install fiftyone
# !pip uninstall --yes opencv-python-headless==4.5.5.62
# !pip install opencv-python-headless==4.5.2.52

In [ ]:
import fiftyone.zoo as foz
import fiftyone as fo


dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="validation",
    image_ids=set(df2['image_id'])
)
session = fo.launch_app(dataset)

dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="train",
    image_ids=set(df3['image_id'])
)
session = fo.launch_app(dataset)